<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/compare_SST_GLO12_one_HURR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare:
- Satellite data product from [OSTIA SST.](https://data.marine.copernicus.eu/product/SST_GLO_SST_L4_NRT_OBSERVATIONS_010_001/services)
- Ocean-reanalysis from [Mercator/Glorys12](https://data.marine.copernicus.eu/product/GLOBAL_ANALYSISFORECAST_PHY_001_024/services)
- _You_ need to encode `User Inputs`.

### Install modules

In [ ]:
!pip install tropycal
!pip install copernicusmarine
!pip install cartopy

In [ ]:
import copernicusmarine
from tropycal import tracks
import xarray as xr
import numpy as np
import pandas as pd

from PIL import Image
import glob as glob

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
#import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

#%matplotlib inline

In [ ]:
def get_cmems_data(dsetID, vNames, lon_start, lon_end, lat_start, lat_end, time_start, time_end):

  data_request = {"dataset_id" : dsetID,
    "longitude" : [lon_start, lon_end],
    "latitude" : [lat_start, lat_end],
    "time" : [time_start, time_end],
    "variables" : vNames}

  cms_data =copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    variables = data_request["variables"])

  return cms_data

## User Inputs

In [ ]:
# Basin and year
myBasin = 'north_atlantic'
year = 2024
hurr_name = "Milton"
time_delta = 5 # days before/after storm
dLon, dLat = [5, 5] # plot extra data outside track bounds (in degrees)
skip_plots = 4 # How many time snapshots to plot between start- end of storm

vMin, vMax, cMap = [298., 308., 'gist_ncar'] # Gulf of Mexico- custom for GLO12 (reanalysis)

var_units = '[K]'
track_color = 'k'
DPI=300

# Copernicus credentials
CMEMS_username, CMEMS_passwd = ["sakella", "HbFPyP9M"]

### Dataset specific info (may change with dataset, you need to fill it in carefully by looking up [CMEMS](https://data.marine.copernicus.eu/products)).

In [ ]:
# CMEMS dataset IDs
dsetID_sst = 'METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2' # OSTIA SST
vNames_sst = ['analysed_sst']

In [ ]:
# Reanalysis GLORYS12
# https://catalogue.marine.copernicus.eu/documents/PUM/CMEMS-GLO-PUM-001-024.pdf
#
# Hourly mean surface (2d) fields: cmems_mod_glo_phy_anfc_0.083deg_PT1H-m

dset_oras = "cmems_mod_glo_phy_anfc_0.083deg_PT1H-m"
var_oras = "thetao"
kelvin_to_degC = 273.15

## Get hurricane track data

In [ ]:
basin = tracks.TrackDataset(basin=myBasin, source='hurdat', include_btk=True, interpolate_data=True)
hurr=basin.get_storm((hurr_name, year))
track_file = str(year)+'_'+hurr_name+'.nc'
hurr.to_xarray().to_netcdf(track_file)
print("\nSaved track info for {}, {} to:\n{}".format(hurr_name, year, track_file))

# Hurricane formation and dissipation dates (yyyymmdd)
print("Hurricane: {},\t year: {}".format(hurr_name, year))
print("Formed on: {},\t dissipated on: {}".
format(hurr.time[0].strftime('%Y-%m-%d'), hurr.time[-1].strftime('%Y-%m-%d')))

t0 = hurr.time[0] - pd.Timedelta(days=time_delta)
t1 = hurr.time[-1] + pd.Timedelta(days=time_delta)

print("\nDownloading satellite data between following dates:")
print(t0.strftime('%Y-%m-%d'), "to", t1.strftime('%Y-%m-%d'))

# We'll read back the saved data (for formatting) and use it here onward
hurr=xr.open_dataset(track_file)

## Get satellite data product

In [ ]:
# set credentials
copernicusmarine.login(username=CMEMS_username, password=CMEMS_passwd)

# download satellite data
sat_data = get_cmems_data(dsetID_sst, vNames_sst,
                        hurr.lon.values.min()-dLon, hurr.lon.values.max()+dLon,
                        hurr.lat.values.min()-dLat, hurr.lat.values.max()+dLat,
                        t0.strftime('%Y-%m-%d'), t1.strftime('%Y-%m-%d'))

sat_data_file = 'OSTIA_L4_' + str(year)+'_'+hurr_name+'.nc'
sat_data.to_netcdf(sat_data_file)
print("\nSaved satellite data for {}, {} to:\n{}".format(hurr_name, year, sat_data_file))

## Get ocean reanalysis data

In [ ]:
lon_s, lon_e = [hurr.lon.values.min()-dLon, hurr.lon.values.max()+dLon]
lat_s, lat_e = [hurr.lat.values.min()-dLat, hurr.lat.values.max()+dLat]
t_s, t_e = [t0.strftime('%Y-%m-%d'), t1.strftime('%Y-%m-%d')]

oras_data_path = 'GLO12_'+ 'sst_' +str(year)+'_'+hurr_name

!copernicusmarine subset --dataset-id $dset_oras --username $CMEMS_username --password $CMEMS_passwd -o $oras_data_path --variable $var_oras --minimum-latitude $lat_s --maximum-latitude $lat_e --minimum-longitude $lon_s --maximum-longitude $lon_e --start-datetime $t_s --end-datetime $t_e --service arco-geo-series

## Plot will be generated for select time slices
- Reanalysis data will be saved for those slices **only**.
- Following depends on `skip_plots` (set by user)

In [ ]:
time_slices_plot = xr.concat([hurr.time[0] - pd.Timedelta(days=time_delta),
                    hurr.time[0:-1:skip_plots],
                    hurr.time[-1] + pd.Timedelta(days=time_delta)], dim='time')
#print(time_slices_plot)

In [ ]:
# Subset reanalysis data for the time slices (to plot)
oras_full_file = sorted( glob.glob(oras_data_path + "*/*.nc"))
#print(oras_full_file)

ds_oras=xr.open_dataset(oras_full_file[0]).sel(time=time_slices_plot, method='nearest')
ds_oras.to_netcdf('subset_' + oras_data_path +'.nc')

## Plot OSTIA SST

In [ ]:
# Create a figure and axes with a specific projection
fig, ax_all = plt.subplots(nrows=1, ncols=len(time_slices_plot), sharey=True, figsize=(12, 10), clear=True, subplot_kw={'projection': ccrs.PlateCarree()})

for ax, time_plot in zip(ax_all, time_slices_plot):
  # Plot the data
  im = sat_data[vNames_sst[0]].sel(time=time_plot, method="nearest").plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, vmin=vMin, vmax=vMax, cmap=cMap)

  # Add features like coastlines, gridlines, etc.
  ax.coastlines()
  ax.gridlines()
  ax.set_extent([lon_s, lon_e, lat_s, lat_e])

  gl = ax.gridlines(draw_labels=True)
  gl.top_labels = False
  gl.right_labels = False
  gl.xformatter = LONGITUDE_FORMATTER
  gl.yformatter = LATITUDE_FORMATTER
  gl.xlabel_style = {'size': 2}
  gl.ylabel_style = {'size': 2}

  # Set the title
  date1 = time_plot.values
  yyyymmdd = time_plot.values.astype('str').split('T')[0]
  ax.set_title('{}'.format(yyyymmdd), fontsize=8)

  # Overlay hurricane when it went by
  if date1 >= hurr.time[0].values and date1 <= hurr.time[-1].values:
      ax.scatter(hurr.sel(time=slice(hurr.time[0], date1)).lon,\
            hurr.sel(time=slice(hurr.time[0], date1)).lat,\
            s=hurr.sel(time=slice(hurr.time[0], date1)).vmax/10., marker='x', c=track_color, alpha=0.4, transform=ccrs.PlateCarree())

# add separate colorbar- deliberately suppress it for visual appeal
cax = fig.add_axes([0.91, 0.45, 0.01, 0.1]) # Create an inset axes
cb = plt.colorbar(im, cax, ticks=np.arange(vMin, vMax, (vMax-vMin)/10), shrink=0.5)
cb.ax.tick_params(labelsize=6)
cb.ax.set_xlabel(var_units, fontsize=12)

# save figure
figName = hurr_name + str(year) + '_ostia_sst.png'
plt.savefig(figName, dpi=DPI, bbox_inches='tight')

## Plot reanalysis SST

In [ ]:
# Create a figure and axes with a specific projection
fig, ax_all = plt.subplots(nrows=1, ncols=len(time_slices_plot), sharey=True, figsize=(12, 10), clear=True, subplot_kw={'projection': ccrs.PlateCarree()})

for ax, time_plot in zip(ax_all, time_slices_plot):
  # Plot the data
  im = (ds_oras[var_oras]+kelvin_to_degC).sel(time=time_plot, method="nearest").plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False,\
  vmin=vMin, vmax=vMax, cmap=cMap)

  # Add features like coastlines, gridlines, etc.
  ax.coastlines()
  ax.gridlines()
  ax.set_extent([lon_s, lon_e, lat_s, lat_e])

  gl = ax.gridlines(draw_labels=True)
  gl.top_labels = False
  gl.right_labels = False
  gl.xformatter = LONGITUDE_FORMATTER
  gl.yformatter = LATITUDE_FORMATTER
  gl.xlabel_style = {'size': 2}
  gl.ylabel_style = {'size': 2}

  # Set the title
  date1 = time_plot.values
  yyyymmdd = time_plot.values.astype('str').split('T')[0]
  ax.set_title('{}'.format(yyyymmdd), fontsize=8)

  # Overlay hurricane when it went by
  if date1 >= hurr.time[0].values and date1 <= hurr.time[-1].values:
      ax.scatter(hurr.sel(time=slice(hurr.time[0], date1)).lon,\
            hurr.sel(time=slice(hurr.time[0], date1)).lat,\
            s=hurr.sel(time=slice(hurr.time[0], date1)).vmax/10., marker='x', c=track_color, alpha=0.4, transform=ccrs.PlateCarree())

# add separate colorbar
cax = fig.add_axes([0.91, 0.45, 0.01, 0.1]) # Create an inset axes
cb = plt.colorbar(im, cax, ticks=np.arange(vMin, vMax, (vMax-vMin)/10), shrink=0.5)
cb.ax.tick_params(labelsize=6)
cb.ax.set_xlabel(var_units, fontsize=12)

# save figure
figName = hurr_name + str(year) + '_GLO12_' + var_oras + '.png'
plt.savefig(figName, dpi=DPI, bbox_inches='tight')